In [ ]:
!pip install pandas openpyxl

In [ ]:
import json
import pandas as pd

legislativas = pd.read_excel("dados/legislativas.xlsx",header=[0,1],sheet_name="Quadro")

In [ ]:
display(legislativas)

In [ ]:

def eleitoresPorto() -> int:
    
    region_name = 'Área Metropolitana do Porto'
    interest_row = legislativas[legislativas[('Territórios', 'Região')] == region_name]
    return interest_row['Total'].max().idxmax()

eleitoresPorto()

In [ ]:
# get the first row of the dataframe
first_row = legislativas["Total"].iloc[0]

# Get the first element from the first row if it is a Series
first_element = first_row.iloc[0] if hasattr(first_row, 'iloc') else "The row is not a Series with multiple elements."
first_element

len(legislativas["Total"].columns)

In [ ]:


def taxaAbstencao() -> list[tuple[int,float]]:

    porcentagem_de_abstencao = []
    abstencao = []

    total = legislativas["Total"].iloc[0]
    eleitores = legislativas["Votantes"].iloc[0]
    porcentagem_de_abstencao.append(((total - eleitores) / total) * 100)

    for index in range(len(legislativas["Total"].columns)):
        abstencao.append((legislativas["Total"].columns[index], porcentagem_de_abstencao[0].iloc[index]))

    return abstencao

taxaAbstencao()

In [ ]:

def perdaGrandesMunicipios() -> dict[str,int]:
    # Por cada município com pelo menos 10000 votantes num dos anos, qual o ano em que mais perderam votantes em relação às eleições anteriores? Complete a definição da função perdaGrandesMunicipios, que retorna um dicionário { municipio : ano }.


    municipios_validos = {}

    # conseguir o a serie pandas com todos os municipios

    location_type = 'Município'
    municipal_rows = legislativas[legislativas[('Territórios', 'Âmbito Geográfico')] == location_type]

    # primeiro achamos todos os municipios com pelo menos 10000 votantes num dos anos
    for row_index, row_serie in municipal_rows["Votantes"].iterrows():
        for year_index in range(len(municipal_rows["Votantes"].columns)):
            if row_serie.iloc[year_index] >= 10000:
                municipios_validos[legislativas['Territórios']['Região'].iloc[row_index]] = ""
                break
    
    # map the list index to the year
    years2index = {index: year for index, year in enumerate(municipal_rows["Votantes"].columns)}

    # agora vamos achar o ano em que mais perderam votantes em relação às eleições anteriores
    for municipio in municipios_validos:

        # find the row index of the municipio
        municipio_row_index = municipal_rows[municipal_rows['Territórios']['Região'] == municipio].index[0]
        # get the pandas series of the municipio
        municipio_data = legislativas["Votantes"].iloc[municipio_row_index]
        # transform the series into a list
        municipio_data_list_raw = municipio_data.tolist()
        # make a copy of the list
        municipio_data_list_shifted = municipio_data_list_raw.copy()
        # shift the list to the right
        municipio_data_list_shifted.insert(0, 0)

        # differences between the years list
        differences = [municipio_data_list_raw[i] - municipio_data_list_shifted[i] for i in range(len(municipio_data_list_raw))]
        # remove the first element of the list
        differences.pop(0)
        # find the index of the year with the biggest difference
        biggest_difference_index = differences.index(min(differences))
        # map the index to the year
        municipios_validos[municipio] = years2index[biggest_difference_index+1]
        # print(municipio_row_index, municipio)
        # print()
        # print(municipio_data_list_raw)
        # print(municipio_data_list_shifted)
        # print(differences)
        # print(biggest_difference_index)
        # print(years2index[biggest_difference_index+1])
        # print()
    
    return municipios_validos

perdaGrandesMunicipios()


In [ ]:
def demografiaMunicipios() -> dict[str,tuple[str,str]]:
    # Por cada região NUTS III, qual o município que mais perdeu e o que mais ganhou eleitores entre 1975 e 2022? Complete a definição da função demografiaMunicipios, que retorna um dicionário { regiao : (municipioPerdeu,municipioGanhou) }.

    location_type = 'NUTS III'
    nuts_rows_df = legislativas[legislativas[('Territórios', 'Âmbito Geográfico')] == location_type]
    #filtar para somente o Total
    nuts_rows = nuts_rows_df["Total"]

    location_type = 'Município'
    municipal_rows = legislativas[legislativas[('Territórios', 'Âmbito Geográfico')] == location_type]


    nuts_municipality = {}
    # { regiao : [] }
    # exemplo: { 'Norte' : [4, 5, 6, 7, 8, 9, 10, 11, 12, 13] }

    for nuts_row_index in range(len(nuts_rows)):
        # list of row indexes that are under the NUTS III category
        # get the first row of the nuts_rows
        # print(nuts_rows.index[nuts_row_index])
        # check if the nuts_row_index is the last element of the list
        if nuts_row_index == len(nuts_rows)-1:
            list_of_nuts = list(range(nuts_rows.index[nuts_row_index]+1, len(legislativas)))
        else:
            list_of_nuts = list(range(nuts_rows.index[nuts_row_index]+1, nuts_rows.index[nuts_row_index+1]))
        # print(list_of_nuts)
        # add this nuts to the dictionary
        nuts_municipality[nuts_rows_df[('Territórios', 'Região')].iloc[nuts_row_index]] = list_of_nuts


    # --- ok --

    # pra cada municipio dentro de cada regiao, é necessario achar o valor da diferenca entre os anos 1975 e 2022 na coluna Total

    municipality_differences = {}
    municipality_data_dict = {}

    results = {}

    print(municipal_rows.head())

    print(municipal_rows.columns[0])

    municipal_rows.set_index(legislativas.columns[0], inplace=True)
    display(municipal_rows)
    # i whant to set the first column as the index
    # municipal_rows.set_index(('Territórios', 'Região'), inplace=True)

    # 1. acessar a primeira regiao
    for region in nuts_municipality:
        print("-------------------")
        # 2. acessar o primeiro municipio
        for municipality_index in nuts_municipality[region]:
            # 3. acessar a serie do municipio
            print(municipality_index, municipal_rows[('Territórios', 'Região')].iloc[municipality_index])
            municipality_data = legislativas["Total"].iloc[municipality_index]
            # 4. transformar a serie em uma lista
            municipality_data_list = municipality_data.tolist()
            # 5. achar a diferenca entre os anos 1975 e 2022
            difference = municipality_data_list[-1] - municipality_data_list[0]
            municipality_data_dict.update({legislativas[('Territórios', 'Região')].iloc[municipality_index]: difference})
        municipality_differences[region] = municipality_data_dict
        municipality_data_dict = {}
    
    for region in municipality_differences:
        print(region)
        print(municipality_differences[region])
        print()
        results[region] = (min(municipality_differences[region], key=municipality_differences[region].get), 
                           max(municipality_differences[region], key=municipality_differences[region].get))


    print(results)
    
    return None

demografiaMunicipios()

# Tarefa 4
---

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd

# Carregar os dados como descrito anteriormente e construir o grafo G
# nominations = pd.read_csv("dados/nominations.csv")
# G = nx.DiGraph()
# Adicionar os dados ao grafo como descrito nas modificações anteriores.

def plot_graph(G):
    pos = nx.spring_layout(G)  # Posicionamento dos nós para uma visualização clara
    plt.figure(figsize=(12, 12))  # Define o tamanho da figura
    nx.draw(G, pos, with_labels=True, node_color='skyblue', node_size=5000, 
            edge_color='k', linewidths=1, font_size=15, arrows=True)
    plt.title('Visualização do Grafo de Nomeações para o Prêmio Nobel')
    plt.show()

In [27]:
nominations = pd.read_csv("dados/nominations.csv")

ramon_rows = nominations[nominations["Nominee(s)"].str.contains("Ramón Menéndez Pidal")]
display(ramon_rows)

unique_nominators = set(ramon_rows["Nominator(s)"].unique())
print(unique_nominators)



,Year,Category,Nominee(s),Nominator(s)
12821,1931,Literature,Ramón Menéndez Pidal,49 members of The Nobel Prize Committee of the...
12822,1931,Literature,Ramón Menéndez Pidal,52 representatives
12823,1931,Literature,Ramón Menéndez Pidal,7 members of Faculté de Philosophie et Lettres...
12824,1931,Literature,Ramón Menéndez Pidal,14 members of the Portuguese Academy of Science
12825,1931,Literature,Ramón Menéndez Pidal,6 members of Real Academia de Bellas Artes de ...
...,...,...,...,...
14778,1965,Literature,Ramón Menéndez Pidal,O Deutschmann
14883,1966,Literature,Ramón Menéndez Pidal,Gunnar Tilander
14884,1966,Literature,Ramón Menéndez Pidal,Henri Guiter
14987,1967,Literature,Ramón Menéndez Pidal,Gunnar Tilander


{'16 members of the Royal Academy of History', 'Gabriel Maura', 'Santiago Ramon y Cajàl', 'Manuel Briones', 'Rafael Lapesa', 'Angel Rosenblat', 'Jesús Castañon', 'Harri Meier\r\n Wilhelm Neuss', ' University of Sevilla (Faculty of Arts)', 'William Nitze', 'O Deutschmann', 'Veikko Väänänen', '14 members', 'John Corominas', ' Salvadoran Academy of language', 'Luis Redonet', 'Paulo de Medeyros', 'Julio Casares', 'Marcel Édouard Bataillon', 'Rudolf Grossmann', 'Edwin Morby', 'S Griswold Morley', 'Alonso Zamora Vicente', 'Erik Staaff', 'Mariano Baquero Goyanes', 'Raimundo Lida', '14 professors', 'Leonida Biancolini', 'Maria Rosa Lida de Malkiel\r\n Yakov Malkiel', 'Concha Zardoya', 'Per Hallström', 'H van Waeyenbergh', 'Knut Hjalmar Leonard Hammarskjöld', 'J Rees', '6 members of Real Academia de Bellas Artes de Toledo', 'H Friedrich', 'Edward Wilson', 'Günther Reichenkron\r\n Fritz Neubert', 'Rafael Benitez Claros', 'Nobumoto Ohama', '12 professors in Spanish\r\n M Ruipérez\r\n Juan Maluque

In [23]:
import pandas as pd
import networkx as nx

# Creating a DataFrame from the provided data
nominations = pd.read_csv("dados/nominations.csv")
nominations = nominations.copy()

display(nominations)


def maisNomeado():
    G = nx.DiGraph()

    # Percorrendo cada linha do DataFrame
    for _, row in nominations.iterrows():
        nominators = [name.strip() for name in row["Nominator(s)"].replace('\r\n', ',').split(',')]
        nominees = [name.strip() for name in row["Nominee(s)"].replace('\r\n', ',').split(',')]
        year = row['Year']

        # Adicionando cada nomeado ao grafo
        for nominee in nominees:
            for nominator in nominators:
                if nominee == "Ramón Menéndez Pidal":
                    if nominator not in data:
                        print(nominator)
                if G.has_edge(nominator, nominee):
                    G[nominator][nominee]['years'].add(year)
                else:
                    G.add_edge(nominator, nominee, years={year})

    # Contagem dos nominadores únicos por nomeado
    nominee_counts = {}
    
    for nominator, nominee in G.edges():
        if nominee not in nominee_counts:
            nominee_counts[nominee] = set()
        nominee_counts[nominee].add(nominator)

    # Encontrar o nomeado com o maior número de nominadores únicos
    most_nominated = None
    max_nominators = 0
    for nominee, nominators in nominee_counts.items():
        if len(nominators) > max_nominators:
            most_nominated = nominee
            max_nominators = len(nominators)

    return most_nominated, max_nominators

maisNomeado()

,Year,Category,Nominee(s),Nominator(s)
0,1901,Physics,Svante Arrhenius,Adolf von Baeyer
1,1901,Physics,Svante Arrhenius,August Sundell
2,1901,Physics,Henri Becquerel,Marcellin Berthelot
3,1901,Physics,William Campbell,George Hale
4,1901,Physics,William Campbell,Simon Newcomb
...,...,...,...,...
20069,1970,Peace,Abbé Pierre,Lars Roar Langslet
20070,1970,Peace,Alfred Verdroß-Drossberg,Josef Klaus
20071,1970,Peace,Hélder Câmara,René Cassin
20072,1970,Peace,Hélder Câmara,6 members of the Dutch parliament


('Ramón Menéndez Pidal', 148)

In [20]:
data = [ "16 members of the Royal Academy of History", "Tomas Navarro", "Emilio Alarcos García", "Luis Suárez Fernández", "Angel del Rio", "John Spagnoli", "Gabriel Maura", "René Taupin", "Santiago Ramon y Cajàl", "M Ruipérez", "Manuel Briones", "Rafael Lapesa", "Angel Rosenblat", "Jesús Castañon", "William Nitze", "Manuel Munoz Cortes", "O Deutschmann", "Maria Rosa Lida de Malkiel", "Veikko Väänänen", "14 members", "John Corominas", "Luis Redonet", "Cuban Academy of Language (Academia Cubana de Lengua)", "Luciano de la Calzada", "Paulo de Medeyros", "Julio Casares", "Marcel Édouard Bataillon", "Rudolf Grossmann", "Edwin Morby", "L.P. Peckham", "S Griswold Morley", "Alonso Zamora Vicente", "E Arnould", "Erik Staaff", "Mariano Baquero Goyanes", "Raimundo Lida", "14 professors", "Leonida Biancolini", "Concha Zardoya", "Per Hallström", "H van Waeyenbergh", "Knut Hjalmar Leonard Hammarskjöld", "J Rees", "6 members of Real Academia de Bellas Artes de Toledo", "H Friedrich", "Edward Wilson", "Fritz Neubert", "Rafael Benitez Claros", "Nobumoto Ohama", "Emilio Orozco Díaz", "Vicente Palacio Atard", "Ernesto Da Cal", "Juan José Martin Gonzalez", "Gunnar Tilander", "Elie Lambert", "Leo Spitzer", "A Mas", "Several professors", "Günther Reichenkron", "Hans Flasche", "Fredrik Norman", "Fritz Krüger", "Martín de Riquer Morera", "9 professors at the faculty of Humanities", "12 professors in Spanish", "Alfredo Kindelán", "Royal Spanish Academy", "Alfonsina Braun", "Walther von Wartburg", "Angelo Monteverdi", "Juan Maluquer de Motes", "Mariano Picon Salas", "A Burger", "Jules Horrent", "Ramón de Abadal", "Vicente Gaos", "Vicente Garcia de Diego", "Alfonso Corral Castanedo", "7 members of Faculté de Philosophie et Lettres", "P Russell", "Yakov Malkiel", "Emiliano Diez Echarri", "Aurelio Vinas", "Edmund Schramm", "Bolivian Academy of Language", "F Battaglia", "Manuel Alvar", "Raymond Willis", "Amalia de del Rio", "Rámon Sugranyes de Franch", "Hispanic Society of America", "Harri Meier", "Albert Sloman", "Walter Pabst", "P Le Gentil", "José Ramon Ayala", "Maír José Benardete", "100 institutions and individuals", "A Parker", "Chilean Academy of Language (Academia Chilena de la Lengua)", "Antonio Gallego Morell", "Wilhelm Kellermann", "E Kohler", "27 members of the Royal Spanish Academy (Real Academia Española )", "Salvadoran Academy of language", "Academie Argentina de Letras", "Juan Lopez-Morillas", "Francisco Yndurain", "Brussels;", "Fukutaro Okui", "E Riley", "30 members of the Royal Spanish Academy (Real Academia Española )", "Member", "Francisco Sanchez Castaner", "H Baader", "Andreas Blinkenberg", "13 members", "Angel Valbuena Prat", "Robert Ricard", "Hans Rheinfelder", "Norman Torrey", "Júlio Dantas", "21 members of the Royal Academy of History", "James Shearer", "R Cantel", "Emilio Alarcos Llorach", "William Atkinson", "Wilhelm Neuss", "49 members of The Nobel Prize Committee of the Society of Authors", "Members of the Spanish Royal Academy (Real Academia Espanola)", "Melchior Fernández Almagro", "Henri Guiter", "Erich Freiherr von Richthofen", "Antonio Llorente Maldonado", "J van Praag", "Joaquin Casalduero", "Erhard Lommatzsch", "Fouad Boustany", "Gaston Gille", "14 members of the Portuguese Academy of Science", "William Smith", "Gerardo Diego", "William Fichter", "University of Sevilla (Faculty of Arts)", "52 representatives", "Khalil Mardam Bey", "Harry Kurz", "Antonio Badia Margarit" ]

len(data)

148

## 2
---

In [30]:
import pandas as pd
import networkx as nx
from collections import defaultdict

# Carrega os dados novamente
nominations = pd.read_csv("dados/nominations.csv")

def nomeacoesCruzadas():
    # Criar o grafo direcionado
    G = nx.DiGraph()

    # Adiciona as arestas baseadas nas nomeações
    for _, row in nominations.iterrows():
        nominators = [name.strip() for name in row["Nominator(s)"].replace('\r\n', ',').split(',')]
        nominees = [name.strip() for name in row["Nominee(s)"].replace('\r\n', ',').split(',')]
        category = row["Category"]
        for nominator in nominators:
            for nominee in nominees:
                # Adiciona uma aresta do nominador para o nomeado
                G.add_edge(nominator, nominee, category=category)
    
    # Encontrar o maior componente fortemente conectado
    largest_scc = max(nx.strongly_connected_components(G), key=len)
    
    # Coletar categorias envolvidas neste componente
    categories = set()
    for u, v, data in G.edges(data=True):
        if u in largest_scc and v in largest_scc:
            categories.add(data['category'])

    # Retorna o tamanho do maior SCC e as categorias envolvidas
    return len(largest_scc), categories

# Chamada da função
print(nomeacoesCruzadas())

(357, {'Peace', 'Chemistry', 'Physics', 'Medicine'})


## 3
---


In [3]:
import pandas as pd
import networkx as nx

# Carregando os dados
nominations = pd.read_csv("dados/nominations.csv")

def caminhoEinsteinFeynman() -> list[str]:

    local_nominations = nominations.copy()
    
    # Filtrar dados para o período entre 1921 e 1965 e para a categoria de Física
    local_nominations = local_nominations[(local_nominations['Year'] >= 1921) & (local_nominations['Year'] <= 1965) & (local_nominations['Category'] == 'Physics')]
    
    # Criando o grafo
    G = nx.DiGraph()
    
    for idx, row in local_nominations.iterrows():
        nominators = [name.strip() for name in row["Nominator(s)"].replace('\r\n', ',').split(',')]
        nominees = [name.strip() for name in row["Nominee(s)"].replace('\r\n', ',').split(',')]
        
        # Adicionando arestas de cada nominador para cada nomeado
        for nominator in nominators:
            for nominee in nominees:
                G.add_edge(nominator, nominee)

    # Encontrando o caminho entre Einstein e Feynman
    # Einstein como nominador (teremos que verificar se ele nomeou alguém diretamente)
    # Feynman como nomeado
    try:
        # Inicialmente assumimos que Einstein e Feynman são parte dos nodos
        # Este método levanta uma exceção se não houver caminho
        path = nx.shortest_path(G, source="Albert Einstein", target="Richard Phillips Feynman")
        return path
    except nx.NetworkXNoPath:
        return []  # Retornar lista vazia se não houver caminho


# Teste da função
caminhoEinsteinFeynman()

['Albert Einstein',
 'James Franck',
 'Emilio Gino Segrè',
 'Richard Phillips Feynman']